# Numba-GPU

codigo baseado em:

https://nyu-cds.github.io/python-numba/05-cuda/

https://numba.pydata.org/numba-doc/dev/cuda/kernels.html

Fila | Wall-clock máximo (em horas) | Número mínimo de nós (núcleos+ dispositivos) | Número máximo de nós (núcleos+ dispositivos) | Número máximo de tarefas em execução por usuário | Número máximo de tarefas em fila por usuário
- | - | - | - | - | -
nvidia_small | 1 | 1 (24+2) | 20 (480+40) | 4 | 24
nvidia_dev | 0:20 | 1 (24+2) | 4 (96+8) | 1 | 1
sequana_gpu | 96 | 1 (48+4) | 21 (1008+84) | 4 | 24


    ssh sdumont18
    Xeon Gold 6152, 22 núcleos, 44 threads, total 88 "cpus", 754 G RAM
    4x Tesla V100
    
    module load sequana/current
    module load sdbase
    module load anaconda3/2018.12
    conda activate --stack /scratch/ampemi/pedro.santos2/anaconda3/envs/dscience
    export NUMBAPRO_NVVM=/usr/local/cuda-10.1/nvvm/lib64/libnvvm.so
    export NUMBAPRO_LIBDEVICE=/usr/local/cuda-10.1/nvvm/libdevice/


In [15]:
! python --version

Python 3.6.12 :: Anaconda, Inc.


In [16]:
import numpy
numpy.version.version

'1.18.1'

In [12]:
! lscpu

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                88
On-line CPU(s) list:   0-87
Thread(s) per core:    2
Core(s) per socket:    22
Socket(s):             2
NUMA node(s):          2
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 85
Model name:            Intel(R) Xeon(R) Gold 6152 CPU @ 2.10GHz
Stepping:              4
CPU MHz:               2101.000
CPU max MHz:           2101.0000
CPU min MHz:           1000.0000
BogoMIPS:              4200.00
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              1024K
L3 cache:              30976K
NUMA node0 CPU(s):     0-21,44-65
NUMA node1 CPU(s):     22-43,66-87
Flags:                 fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc art arch_perfmon pebs bts rep_good nopl 

In [14]:
! cat /proc/meminfo | grep MemTotal

MemTotal:       791009752 kB


In [1]:
! nvidia-smi

Mon Nov 30 19:19:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.00    Driver Version: 418.87.00    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   49C    P0    30W / 250W |    125MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  Off  | 00000000:5E:00.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |     15MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [1]:
# verificando se Cuda está ativo
from numba import cuda
print(cuda.gpus)

<Managed Device 0>, <Managed Device 1>, <Managed Device 2>, <Managed Device 3>


In [11]:
import math
from time import time
import numpy as np
from numba import cuda, jit, prange

@cuda.jit
def st3(a1, a2):
    n = a1.shape[0] - 1
    i, j = cuda.grid(2)
    if (i > 0 and j > 0) and (i < n and j < n) :
        a1[i,j] = a2[i,j]/2.0+(a2[i-1,j]+a2[i+1,j]+a2[i,j-1]+a2[i,j+1])/8.0

def calc3(anew, aold, heat, sizeEnd, niters, nsources, sources, energy,
          blocks_per_grid, threads_per_block):
    for iters in range(0, niters, 2):
        st3[blocks_per_grid, threads_per_block](anew, aold)
        for i in range(0, nsources) :
            anew[sources[i,0], sources[i,1]] += energy    # heat source
        st3[blocks_per_grid, threads_per_block](aold, anew)
        for i in range(0, nsources):  
            aold[sources[i,0], sources[i,1]] += energy    # heat source        

#def par_cuda():
n            = 4800    # nxn grid
energy       = 1       # energy to be injected per iteration
niters       = 500     # number of iterations
nsources     = 3       # sources of energy
size         = n + 2   # plus the ghost zone
sizeEnd      = n + 1

# initialize the data arrays
anew         = np.zeros((size, size), np.float64)
aold         = np.zeros((size, size), np.float64)
# initialize three heat sources
sources      = np.empty((3,2), np.int32)
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]
heat         = 0       # system total heat sum

# copy the arrays to the device
anew_global_mem = cuda.to_device(anew)
aold_global_mem = cuda.to_device(aold)

# configure blocks & grids
# set the number of threads in a block
threads_per_block = (8, 8)
# calculate the number of thread blocks in the grid
blocks_per_grid_x = math.ceil(aold.shape[0] / threads_per_block[0])
blocks_per_grid_y = math.ceil(aold.shape[1] / threads_per_block[1])
blocks_per_grid   = (blocks_per_grid_x, blocks_per_grid_y)

t = time()
# main calc
calc3(anew_global_mem, aold_global_mem, heat,
    sizeEnd, niters, nsources, sources, energy,
    blocks_per_grid, threads_per_block)

# copy the result back to the host
aold = aold_global_mem.copy_to_host()

for j in range(1, sizeEnd):
    for i in range(1, sizeEnd):
        heat = heat + aold[i,j]
t = time() - t

# show the result if desired
print("Heat=%.4f | Tempo=%.4f" % (heat, t))

Heat=1500.0000 | Tempo=8.0218


REV2, grava um arquivo:

In [2]:
%%writefile sngr2.py
import math
from time import time
import numpy as np
from numba import cuda, jit, prange

@cuda.jit
def st3(a1, a2):
    n = a1.shape[0] - 1
    i, j = cuda.grid(2)
    if (i > 0 and j > 0) and (i < n and j < n) :
        a1[i,j] = a2[i,j]/2.0+(a2[i-1,j]+a2[i+1,j]+a2[i,j-1]+a2[i,j+1])/8.0

def calc3(anew, aold, heat, sizeEnd, niters, nsources, sources, energy,
          blocks_per_grid, threads_per_block):
    for iters in range(0, niters, 2):
        st3[blocks_per_grid, threads_per_block](anew, aold)
        for i in range(0, nsources) :
            anew[sources[i,0], sources[i,1]] += energy    # heat source
        st3[blocks_per_grid, threads_per_block](aold, anew)
        for i in range(0, nsources):  
            aold[sources[i,0], sources[i,1]] += energy    # heat source        

#def par_cuda():
n            = 4800    # nxn grid
energy       = 1       # energy to be injected per iteration
niters       = 500     # number of iterations
nsources     = 3       # sources of energy
size         = n + 2   # plus the ghost zone
sizeEnd      = n + 1

# initialize the data arrays
anew         = np.zeros((size, size), np.float64)
aold         = np.zeros((size, size), np.float64)
# initialize three heat sources
sources      = np.empty((3,2), np.int32)
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]
heat         = 0       # system total heat sum

# copy the arrays to the device
anew_global_mem = cuda.to_device(anew)
aold_global_mem = cuda.to_device(aold)

# configure blocks & grids
# set the number of threads in a block
threads_per_block = (8, 8)
# calculate the number of thread blocks in the grid
blocks_per_grid_x = math.ceil(aold.shape[0] / threads_per_block[0])
blocks_per_grid_y = math.ceil(aold.shape[1] / threads_per_block[1])
blocks_per_grid   = (blocks_per_grid_x, blocks_per_grid_y)

t = time()
# main calc
calc3(anew_global_mem, aold_global_mem, heat,
    sizeEnd, niters, nsources, sources, energy,
    blocks_per_grid, threads_per_block)

# copy the result back to the host
aold = aold_global_mem.copy_to_host()

for j in range(1, sizeEnd):
    for i in range(1, sizeEnd):
        heat = heat + aold[i,j]
t = time() - t

# show the result if desired
print("Heat=%.4f | Tempo=%.4f" % (heat, t))

Writing sngr2.py


## REV2 : rodando 4x no sdumont18

- ambiente antigo não funciona mais, rodando agora com:
```
SCR=/scratch${PWD#/prj}
source $SCR/env2/etc/profile.d/conda.sh
conda activate $SCR/env2
conda activate --stack $SCR/env3
```

In [6]:
%%bash
python sngr2.py
python sngr2.py
python sngr2.py
python sngr2.py

Heat=1500.0000 | Tempo=6.6508
Heat=1500.0000 | Tempo=6.7153
Heat=1500.0000 | Tempo=6.5470
Heat=1500.0000 | Tempo=6.5648


In [13]:
%%bash
gfortran --version | head -1
mpiexec --version | head -2
conda list '^(anaconda|python)$'
cd
source /scratch${PWD#/prj}/env2/etc/profile.d/conda.sh
conda deactivate
conda list '^(anaconda|python)$'

GNU Fortran (GCC) 4.8.5 20150623 (Red Hat 4.8.5-39)
HYDRA build details:
    Version:                                 3.3.2
# packages in environment at /scratch/ampemi/eduardo.miranda2/env3:
#
# Name                    Version                   Build  Channel
python                    3.9.4                hdb3f193_0  
# packages in environment at /scratch/ampemi/eduardo.miranda2/env2:
#
# Name                    Version                   Build  Channel
anaconda                  2020.11                  py38_0  
python                    3.8.5                h7579374_1  


# Rodando em um B715

In [14]:
%%writefile st-nu-gpu.py
import math
from time import time
import numpy as np
from numba import cuda, jit, prange

@cuda.jit
def st3(a1, a2):
    n = a1.shape[0] - 1
    i, j = cuda.grid(2)
    if (i > 0 and j > 0) and (i < n and j < n) :
        a1[i,j] = a2[i,j]/2.0+(a2[i-1,j]+a2[i+1,j]+a2[i,j-1]+a2[i,j+1])/8.0

def calc3(anew, aold, heat, sizeEnd, niters, nsources, sources, energy,
          blocks_per_grid, threads_per_block):
    for iters in range(0, niters, 2):
        st3[blocks_per_grid, threads_per_block](anew, aold)
        for i in range(0, nsources) :
            anew[sources[i,0], sources[i,1]] += energy    # heat source
        st3[blocks_per_grid, threads_per_block](aold, anew)
        for i in range(0, nsources):  
            aold[sources[i,0], sources[i,1]] += energy    # heat source        

#def par_cuda():
n            = 4800    # nxn grid
energy       = 1       # energy to be injected per iteration
niters       = 500     # number of iterations
nsources     = 3       # sources of energy
size         = n + 2   # plus the ghost zone
sizeEnd      = n + 1

# initialize the data arrays
anew         = np.zeros((size, size), np.float64)
aold         = np.zeros((size, size), np.float64)
# initialize three heat sources
sources      = np.empty((3,2), np.int32)
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]
heat         = 0       # system total heat sum

# copy the arrays to the device
anew_global_mem = cuda.to_device(anew)
aold_global_mem = cuda.to_device(aold)

# configure blocks & grids
# set the number of threads in a block
threads_per_block = (8, 8)
# calculate the number of thread blocks in the grid
blocks_per_grid_x = math.ceil(aold.shape[0] / threads_per_block[0])
blocks_per_grid_y = math.ceil(aold.shape[1] / threads_per_block[1])
blocks_per_grid   = (blocks_per_grid_x, blocks_per_grid_y)

t = time()
# main calc
calc3(anew_global_mem, aold_global_mem, heat,
    sizeEnd, niters, nsources, sources, energy,
    blocks_per_grid, threads_per_block)

# copy the result back to the host
aold = aold_global_mem.copy_to_host()

for j in range(1, sizeEnd):
    for i in range(1, sizeEnd):
        heat = heat + aold[i,j]
t = time() - t

# show the result if desired
print("Heat=%.4f | Tempo=%.4f" % (heat, t))

Overwriting st-nu-gpu.py


In [15]:
%%bash
a='/ampemi/eduardo.miranda2/stnc/Numba'
s='/prj'$a
d='/scratch'$a
cp  $s/st-nu-gpu.py  $d

In [24]:
%%writefile st-nu-gpu.srm
#!/bin/bash
# limites das filas (1,0 UA):
#   nvidia_dev: 0:20, 1 (24+2), 4 (96+8), 1, 1

#SBATCH --ntasks=01            #Total de tarefas
#SBATCH -J stnugpu             #Nome do job, 8 caracteres
#SBATCH -p nvidia_dev           #Fila (partition) a ser utilizada
#SBATCH --time=00:02:00        #Tempo max. de execução 5 minutos
#SBATCH --exclusive            #Utilização exclusiva dos nós
# #SBATCH --nodes=1              #Qtd de nós
# #SBATCH --ntasks-per-node=1    #Qtd de tarefas por nó ($SLURM_NTASKS_PER_NODE)

echo '========================================'
echo '- Job ID:' $SLURM_JOB_ID
echo '- Tarefas por no:' $SLURM_NTASKS_PER_NODE
echo '- Qtd. de nos:' $SLURM_JOB_NUM_NODES
echo '- Tot. de tarefas:' $SLURM_NTASKS
echo '- Nos alocados:' $SLURM_JOB_NODELIST
nodeset -e $SLURM_JOB_NODELIST
echo '- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):'
echo $SLURM_SUBMIT_DIR
cd $SLURM_SUBMIT_DIR

#echo '-- lscpu ------------------------------'
#lscpu
#echo '-- meminfo -------------------------'
#cat /proc/meminfo | grep MemTotal
#echo '-- nvidia-smi ----------------------'
#nvidia-smi

#Entra no diretório de trabalho
d='/scratch/ampemi/eduardo.miranda2/stnc/Numba'
cd $d

#Modulos
#211217: não funciona mais
#module load anaconda3/2018.12
#conda activate --stack /scratch/ampemi/pedro.santos2/anaconda3/envs/dscience
#export NUMBAPRO_NVVM=/usr/local/cuda-10.1/nvvm/lib64/libnvvm.so
#export NUMBAPRO_LIBDEVICE=/usr/local/cuda-10.1/nvvm/libdevice/
DIR=$PWD
cd
SCR=/scratch${PWD#/prj}
source $SCR/env2/etc/profile.d/conda.sh
conda activate $SCR/env2
conda activate --stack $SCR/env3
cd $DIR

#Executavel
EXEC='python st-nu-gpu.py'

#Dispara a execucao
echo '-- srun -------------------------------'
echo '$ srun -n' $SLURM_NTASKS $EXEC
srun -n $SLURM_NTASKS $EXEC
echo '-- FIM --------------------------------'

Overwriting st-nu-gpu.srm


In [5]:
! sbatch st-nu-gpu.srm

Submitted batch job 787030


In [10]:
! squeue -n stnugpu

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [11]:
%%bash
d='/scratch/ampemi/eduardo.miranda2/stnc/Numba'
cat $d/slurm-787030.out

- Job ID: 787030
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont3080
sdumont3080
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Numba
-- lscpu ------------------------------
Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                24
On-line CPU(s) list:   0-23
Thread(s) per core:    1
Core(s) per socket:    12
Socket(s):             2
NUMA node(s):          2
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 62
Model name:            Intel(R) Xeon(R) CPU E5-2695 v2 @ 2.40GHz
Stepping:              4
CPU MHz:               2400.146
CPU max MHz:           2400.0000
CPU min MHz:           1200.0000
BogoMIPS:              4799.77
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              30720K
NUMA node0 CPU(s):     0-11
NUMA node1 CPU(s

# Usando 16x16

In [12]:
%%writefile st-nu-gpu.py
import math
from time import time
import numpy as np
from numba import cuda, jit, prange

@cuda.jit
def st3(a1, a2):
    n = a1.shape[0] - 1
    i, j = cuda.grid(2)
    if (i > 0 and j > 0) and (i < n and j < n) :
        a1[i,j] = a2[i,j]/2.0+(a2[i-1,j]+a2[i+1,j]+a2[i,j-1]+a2[i,j+1])/8.0

def calc3(anew, aold, heat, sizeEnd, niters, nsources, sources, energy,
          blocks_per_grid, threads_per_block):
    for iters in range(0, niters, 2):
        st3[blocks_per_grid, threads_per_block](anew, aold)
        for i in range(0, nsources) :
            anew[sources[i,0], sources[i,1]] += energy    # heat source
        st3[blocks_per_grid, threads_per_block](aold, anew)
        for i in range(0, nsources):  
            aold[sources[i,0], sources[i,1]] += energy    # heat source        

#def par_cuda():
n            = 4800    # nxn grid
energy       = 1       # energy to be injected per iteration
niters       = 500     # number of iterations
nsources     = 3       # sources of energy
size         = n + 2   # plus the ghost zone
sizeEnd      = n + 1

# initialize the data arrays
anew         = np.zeros((size, size), np.float64)
aold         = np.zeros((size, size), np.float64)
# initialize three heat sources
sources      = np.empty((3,2), np.int32)
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]
heat         = 0       # system total heat sum

# copy the arrays to the device
anew_global_mem = cuda.to_device(anew)
aold_global_mem = cuda.to_device(aold)

# configure blocks & grids
# set the number of threads in a block
threads_per_block = (16, 16)
# calculate the number of thread blocks in the grid
blocks_per_grid_x = math.ceil(aold.shape[0] / threads_per_block[0])
blocks_per_grid_y = math.ceil(aold.shape[1] / threads_per_block[1])
blocks_per_grid   = (blocks_per_grid_x, blocks_per_grid_y)

t = time()
# main calc
calc3(anew_global_mem, aold_global_mem, heat,
    sizeEnd, niters, nsources, sources, energy,
    blocks_per_grid, threads_per_block)

# copy the result back to the host
aold = aold_global_mem.copy_to_host()

for j in range(1, sizeEnd):
    for i in range(1, sizeEnd):
        heat = heat + aold[i,j]
t = time() - t

# show the result if desired
print("Heat=%.4f | Tempo=%.4f" % (heat, t))

Overwriting st-nu-gpu.py


In [13]:
! sbatch st-nu-gpu.srm

Submitted batch job 787034


In [14]:
! squeue -n stnugpu

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            787034 nvidia_de  stnugpu eduardo.  R       0:00      1 sdumont3080


In [19]:
! squeue -n stnugpu

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [20]:
%%bash
d='/scratch/ampemi/eduardo.miranda2/stnc/Numba'
cat $d/slurm-787034.out

- Job ID: 787034
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont3080
sdumont3080
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Numba
-- lscpu ------------------------------
Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                24
On-line CPU(s) list:   0-23
Thread(s) per core:    1
Core(s) per socket:    12
Socket(s):             2
NUMA node(s):          2
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 62
Model name:            Intel(R) Xeon(R) CPU E5-2695 v2 @ 2.40GHz
Stepping:              4
CPU MHz:               2400.146
CPU max MHz:           2400.0000
CPU min MHz:           1200.0000
BogoMIPS:              4799.77
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              30720K
NUMA node0 CPU(s):     0-11
NUMA node1 CPU(s

## 211217 rodando 4x no B715

In [17]:
! sbatch st-nu-gpu.srm

Submitted batch job 10350748


In [18]:
! squeue -n stnugpu --format "%.19S  %.8i  %.10P %.5j  %.2t %.5M %.5D %.4C"

         START_TIME     JOBID   PARTITION  NAME  ST  TIME NODES CPUS
2021-12-17T18:09:57  10350748  nvidia_dev stnug   R  0:04     1   24


In [22]:
! squeue -n stnugpu --format "%.19S  %.8i  %.10P %.5j  %.2t %.5M %.5D %.4C"

         START_TIME     JOBID   PARTITION  NAME  ST  TIME NODES CPUS


In [23]:
! cat /scratch${PWD#/prj}/slurm-10350748.out

- Job ID: 10350748
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont3091
sdumont3091
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Numba
-- lscpu ------------------------------
Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                24
On-line CPU(s) list:   0-23
Thread(s) per core:    1
Core(s) per socket:    12
Socket(s):             2
NUMA node(s):          2
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 62
Model name:            Intel(R) Xeon(R) CPU E5-2695 v2 @ 2.40GHz
Stepping:              4
CPU MHz:               2088.134
CPU max MHz:           2400,0000
CPU min MHz:           1200,0000
BogoMIPS:              4800.12
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              30720K
NUMA node0 CPU(s):     0-11
NUMA node1 CPU

In [25]:
! sbatch st-nu-gpu.srm

Submitted batch job 10350749


In [26]:
! squeue -n stnugpu --format "%.19S  %.8i  %.10P %.5j  %.2t %.5M %.5D %.4C"

         START_TIME     JOBID   PARTITION  NAME  ST  TIME NODES CPUS
2021-12-17T18:13:04  10350749  nvidia_dev stnug   R  0:00     1   24


In [29]:
! squeue -n stnugpu --format "%.19S  %.8i  %.10P %.5j  %.2t %.5M %.5D %.4C"

         START_TIME     JOBID   PARTITION  NAME  ST  TIME NODES CPUS


In [30]:
! cat /scratch${PWD#/prj}/slurm-10350749.out

- Job ID: 10350749
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont3091
sdumont3091
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Numba
-- srun -------------------------------
$ srun -n 1 python st-nu-gpu.py
Heat=1500.0000 | Tempo=17.3833
-- FIM --------------------------------


In [31]:
! sbatch st-nu-gpu.srm

Submitted batch job 10350751


In [32]:
! squeue -n stnugpu --format "%.19S  %.8i  %.10P %.5j  %.2t %.5M %.5D %.4C"

         START_TIME     JOBID   PARTITION  NAME  ST  TIME NODES CPUS
2021-12-17T18:14:07  10350751  nvidia_dev stnug   R  0:00     1   24


In [35]:
! squeue -n stnugpu --format "%.19S  %.8i  %.10P %.5j  %.2t %.5M %.5D %.4C"

         START_TIME     JOBID   PARTITION  NAME  ST  TIME NODES CPUS


In [36]:
! cat /scratch${PWD#/prj}/slurm-10350751.out

- Job ID: 10350751
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont3091
sdumont3091
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Numba
-- srun -------------------------------
$ srun -n 1 python st-nu-gpu.py
Heat=1500.0000 | Tempo=21.7487
-- FIM --------------------------------


In [37]:
! sbatch st-nu-gpu.srm

Submitted batch job 10350772


In [38]:
! squeue -n stnugpu --format "%.19S  %.8i  %.10P %.5j  %.2t %.5M %.5D %.4C"

         START_TIME     JOBID   PARTITION  NAME  ST  TIME NODES CPUS
2021-12-17T18:15:27  10350772  nvidia_dev stnug   R  0:00     1   24


In [40]:
! squeue -n stnugpu --format "%.19S  %.8i  %.10P %.5j  %.2t %.5M %.5D %.4C"

         START_TIME     JOBID   PARTITION  NAME  ST  TIME NODES CPUS


In [41]:
! cat /scratch${PWD#/prj}/slurm-10350772.out

- Job ID: 10350772
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont3091
sdumont3091
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Numba
-- srun -------------------------------
$ srun -n 1 python st-nu-gpu.py
Heat=1500.0000 | Tempo=16.1785
-- FIM --------------------------------


# Usando 32x32 thread block size
não funcionou)

In [1]:
%%writefile st-nu-gpu.py
import math
from time import time
import numpy as np
from numba import cuda, jit, prange

@cuda.jit
def st3(a1, a2):
    n = a1.shape[0] - 1
    i, j = cuda.grid(2)
    if (i > 0 and j > 0) and (i < n and j < n) :
        a1[i,j] = a2[i,j]/2.0+(a2[i-1,j]+a2[i+1,j]+a2[i,j-1]+a2[i,j+1])/8.0

def calc3(anew, aold, heat, sizeEnd, niters, nsources, sources, energy,
          blocks_per_grid, threads_per_block):
    for iters in range(0, niters, 2):
        st3[blocks_per_grid, threads_per_block](anew, aold)
        for i in range(0, nsources) :
            anew[sources[i,0], sources[i,1]] += energy    # heat source
        st3[blocks_per_grid, threads_per_block](aold, anew)
        for i in range(0, nsources):  
            aold[sources[i,0], sources[i,1]] += energy    # heat source        

#def par_cuda():
n            = 4800    # nxn grid
energy       = 1       # energy to be injected per iteration
niters       = 500     # number of iterations
nsources     = 3       # sources of energy
size         = n + 2   # plus the ghost zone
sizeEnd      = n + 1

# initialize the data arrays
anew         = np.zeros((size, size), np.float64)
aold         = np.zeros((size, size), np.float64)
# initialize three heat sources
sources      = np.empty((3,2), np.int32)
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]
heat         = 0       # system total heat sum

# copy the arrays to the device
anew_global_mem = cuda.to_device(anew)
aold_global_mem = cuda.to_device(aold)

# configure blocks & grids
# set the number of threads in a block
threads_per_block = (32, 32)
# calculate the number of thread blocks in the grid
blocks_per_grid_x = math.ceil(aold.shape[0] / threads_per_block[0])
blocks_per_grid_y = math.ceil(aold.shape[1] / threads_per_block[1])
blocks_per_grid   = (blocks_per_grid_x, blocks_per_grid_y)

t = time()
# main calc
calc3(anew_global_mem, aold_global_mem, heat,
    sizeEnd, niters, nsources, sources, energy,
    blocks_per_grid, threads_per_block)

# copy the result back to the host
aold = aold_global_mem.copy_to_host()

for j in range(1, sizeEnd):
    for i in range(1, sizeEnd):
        heat = heat + aold[i,j]
t = time() - t

# show the result if desired
print("Heat=%.4f | Tempo=%.4f" % (heat, t))

Overwriting st-nu-gpu.py


In [2]:
! sbatch st-nu-gpu.srm

Submitted batch job 787898


In [3]:
! squeue -n stnugpu

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            787898 nvidia_de  stnugpu eduardo.  R       0:03      1 sdumont3030


In [9]:
! squeue -n stnugpu

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [10]:
%%bash
d='/scratch/ampemi/eduardo.miranda2/stnc/Numba'
cat $d/slurm-787898.out

- Job ID: 787898
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont3030
sdumont3030
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Numba
-- lscpu ------------------------------
Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                24
On-line CPU(s) list:   0-23
Thread(s) per core:    1
Core(s) per socket:    12
Socket(s):             2
NUMA node(s):          2
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 62
Model name:            Intel(R) Xeon(R) CPU E5-2695 v2 @ 2.40GHz
Stepping:              4
CPU MHz:               2400.146
CPU max MHz:           2400.0000
CPU min MHz:           1200.0000
BogoMIPS:              4800.39
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              30720K
NUMA node0 CPU(s):     0-11
NUMA node1 CPU(s